In [1]:
import sys
sys.path.append("..")

In [2]:
from solvers.MMR.PQK_solver import *
from circuits.circuits import *
from utils.rbf_kernel_tools import *
from squlearn.encoding_circuit import *

In [3]:
from squlearn.observables import SinglePauli
import numpy as np


def PQK_observable(num_qubits, measurement = "XYZ"):
    """"
    Returns the observable for the PQK solver

    Args:

    num_qubits (int): number of qubits in the system
    measurement (str): measurement operator to be applied to the qubits (default: "XYZ")

    Returns:
    _measurement (list): list of SinglePauli objects representing the measurement operator (shape: num_qubits*len(measurement))

    """
    if isinstance(measurement, str):
                _measurement = []
                for m_str in measurement:
                    if m_str not in ("X", "Y", "Z"):
                        raise ValueError("Unknown measurement operator: {}".format(m_str))
                    for i in range(num_qubits):
                        _measurement.append(SinglePauli(num_qubits, i, op_str=m_str))
    return _measurement

In [4]:
num_qubits = 2
num_features = 1
num_samples = 10
num_layers = 3

sigma = 1
measurement = "XYZ"
_measurement = PQK_observable(num_qubits, measurement)
print(len(_measurement), num_qubits*len(measurement))

6 6


In [5]:
from utils.rbf_kernel_tools import analytical_derivative_rbf_kernel, analytical_derivative_dot_kernel_2, matrix_rbf_dxdy_slow
from utils.rbf_kernel_tools import matrix_rbf, matrix_rbf_dx_slow, matrix_rbf_dxdx_slow

We will show that the kernel derivatives calculated by squlearn are the same as the ones analytically calculated for the separable rx fmap

In [6]:
from circuits.circuits import Separable_rx

In [7]:
sigma = 1
gamma = 1/(2*sigma**2)

experiment = { "sigma": sigma}
experiment["circuit_information"] = {"encoding_circuit": Separable_rx, "num_qubits": 1, "num_layers":1}
pqk =PQK_solver(experiment["circuit_information"],
                                Executor("pennylane"), 
                                envelope={"function": matrix_rbf, 
                                            "derivative_function": matrix_rbf_dx_slow, 
                                            "second_derivative_function": matrix_rbf_dxdx_slow,
                                            "mixed_derivative_function": matrix_rbf_dxdy_slow,
                                            "sigma": experiment["sigma"]})

In [8]:
PQK_qnn = pqk.PQK_QNN()
obs_coef = []
X = [0.1, -0.41]
#X = [-0.41, 0.1]
K_f, K_dfdx, K_dfdxdx = pqk.get_PQK_kernel_derivatives(X, PQK_qnn, obs_coef, [1, 2])

[0.1, -0.41]


In [95]:
import sympy as sp

x,y,gamma_sp = sp.symbols("x y gamma")

K = sp.exp(-gamma_sp*(2-2*sp.cos(x-y)))
Kx = sp.diff(K, x)
Kxx = sp.diff(Kx, x) #I will use this expression, the derivation can be seen in the old notebooks 

#or in matrix notation: 

def K_separable_rx_PQK_(X,Y, gamma):
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))
    return gram_matrix

def K_separable_rx_PQK_dx(X,Y, gamma):
    #-2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)

    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = -2*gamma*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.sin(X[i]-Y[j])
    return gram_matrix 

def K_separable_rx_PQK_dxdx(X,Y, gamma):
    #4*gamma**2*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)**2 - 2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*cos(x - y)
    #2*gamma*(2*gamma*sin(x)**2 - 4*gamma*sin(x)*sin(y)*cos(x - y) + 2*gamma*sin(y)**2 - cos(x - y))*exp(-gamma*(2 - 2*cos(x - y)))
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = 2*gamma*(2*gamma*np.sin(X[i])**2 - 4*gamma*np.sin(X[i])*np.sin(Y[j])*np.cos(X[i]-Y[j]) + 2*gamma*np.sin(Y[j])**2 - np.cos(X[i]-Y[j]))*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))
    return gram_matrix 

In [28]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')
x3, y3 = sp.symbols('x_3 y_3')
O1 = sp.symbols('O_1', cls=sp.Function)
O2 = sp.symbols('O_2', cls=sp.Function)
O3 = sp.symbols('O_3', cls=sp.Function)


In [92]:
k = sp.symbols('k', cls=sp.Function)
pqk_fun =  k(O1(x), O2(x), O3(x), O1(y), O2(y), O3(y))
pqk_fun_dx = sp.diff(pqk_fun, x).simplify()
pqk_fun_dxdx = sp.diff(pqk_fun_dx, x).simplify()

Fundamentally, we implement the below equation using einsum notation

In [94]:
pqk_fun_dxdx.simplify()

Derivative(O_1(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2)) + 2*Derivative(O_1(x), x)*Derivative(O_2(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_2(x)) + 2*Derivative(O_1(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_3(x)) + Derivative(O_1(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x)) + Derivative(O_2(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_2(x), 2)) + 2*Derivative(O_2(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x), O_3(x)) + Derivative(O_2(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x)) + Derivative(O_3(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2)) + Derivative(O_3(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_3(x))

In [96]:
from squlearn.kernel.matrix import ProjectedQuantumKernel

In [43]:
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1), executor=Executor("pennylane"), gamma=gamma)

In [44]:
pqk_squlearn.evaluate(X, X)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [97]:
K_separable_rx_PQK_(X,X, gamma)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [98]:
K_f

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [99]:
K_separable_rx_PQK_dx(X,X,gamma)

array([[-0.       , -0.4298443],
       [ 0.4298443, -0.       ]])

In [100]:
K_dfdx

array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [103]:
K_separable_rx_PQK_dxdx(X,X,gamma)

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [104]:
K_dfdxdx

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])